[Notebook of task](https://github.com/DistributedSystemsGroup/Algorithmic-Machine-Learning/blob/master/Challenges/Anomaly_Detection/anomaly_detection_challenge.ipynb)

In [2]:
# Package for scalable bayesian rule lists
!pip3 install --user 'fim'
!pip3 install --user 'pysbrl'

    100% |################################| 348kB 1.2MB/s 
  Running setup.py bdist_wheel for fim ... done
  Stored in directory: /mnt/workspace/.cache/pip/wheels/5c/1c/94/b96c6b9a2eb858e26a675f86a908abfa53a593185b1c058823
Successfully built fim
You are using pip version 18.0, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Elementary
import os
import sys
import re
import random
import matplotlib
import implicit
import warnings
from tqdm import tqdm

# For elementary data manipulation
import pandas as pd
import numpy as np

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For scalable bayesian rule lists
import pysbrl

# Import dataframe and cast names, datatypes and NaNs
from names import column_names, labels
basepath = "/mnt/datasets/anomaly/"
dataDF = pd.read_csv(basepath + 'data.csv', delimiter=";", header=None, names=column_names)
puredataDF = dataDF.drop(labels, axis=1)
anomaliesDF = dataDF.filter(labels, axis=1) 

anomaliesDF_withoutNA = anomaliesDF.fillna(0) # Fill NaNs with 0s, considering them as "not an anomaly"
anomaliesDF_with_negNA = anomaliesDF.fillna(-1) # Fill NaNs with -1 considering them as a separate class for the classifier.

In [21]:
check1DF = anomaliesDF['Check1']
indices_of_zero_elements = list(check1DF[check1DF == 0].index)
indices_of_nan_elements = list(check1DF[check1DF.isna()].index)
print(len(indices_of_nan_elements))

24511


Nyttige artikler om stratified shuffle split
[1](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)
[2](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py)

In [ ]:
def train_val_test_split(train_size=0.6, val_size=0.2, with_nan=False, check, label_dataframe=anomaliesDF):
    test_size = 1 - train_size - test_size
    if check == 'all':
        # perform split for all categories
        pass
    else:
        check_series = label_dataframe[check]
        indices_of_zero_elements = list(check_series[check_series == 0].index)
        indices_of_one_elements = list(check_series[check_series == 1].index)
        
        if with_nan == True:
            indices_of_nan_elements = list(check_series[check_series.isna()].index)

In [ ]:
# Save dataframes to the format the SBRL library requires

In [ ]:
# Use SBRL Library from https://github.com/myaooo/pysbrl
rule_ids, outputs, rule_strings = pysbrl.train_sbrl("data/ttt_train.out", 
                                                    "data/ttt_train.label", 
                                                    20.0, 
                                                    eta=2.0, 
                                                    max_iters=2000, 
                                                    nchain=10, 
                                                    alphas=[1,1])